In this notebook we suggest an approach to estimate future water levels of multiple waterbodies in northern and central Italy. Nine different waterbodies of 4 distinct types (acquifers, water springs, rivers & lakes) are studies, each with different target variables. Instead of aiming for the lowest possible error black box models, we aim to generate interpretable models that do not only give usable estimations, but also allow domain experts to interact with the models. External weather data is collected surrounding the waterbody locations, and we use simple dimension reduction techniques to engineer easy-to-interpret features. Those features are then used as inputs for rolling Gradient Boosted Tree (GBM) models, which fit the natural processes of water bodies quite well (@thomas: referentie). We aim for this notebook to be concise and to the point and will therefor not extensively analyse each of the 20 target measurements or all of the input parameters, but leave the reader with documented code to further dive into points of his or her interest.

# Executive summary

# Data gathering, cleaning and description 

Waterbodies are relatively simple to explain systems: the water level in the future depends on the current water level + the water that comes in - everything that goes out. What comes in and goes out is largely dependent on the weather: rain and evaporation interacting with the way the waterbody works. A river for instance will quickly collect rainwater, while a deeply buried aquifer might take a number of days. The usage of weather predictions are explicitely prohibited for this competition, so instead we aim to get a more detailed view of the current weather by collecting weather data from NASA's POWER system (https://power.larc.nasa.gov/) as external dataset. Based on the given names of the waterbodies, 57 locations of interest are determined, for each which 16 weather metrics (rain, humidity, temperature and wind speeds) are collected in a daily timeframe going back to 2000. We combine this external dataset with the given datasets and manually clean up a number inconsistencies in the data, the details of which are commented in the code. For the features, missing values are filled with a quadratic interpolation, the missing values of the target variables are left empty so we don't train models on guessed measurements, only on actual data. Since water levels are gamma-like distributed (@thomas reference) and can see significant changes due to weather and human interventions, we only clean the obviously faulty data such as 0-values, but remove no further outliers.

# Exploratory data analysis & feature engineering




## Feature engineering

We extract

# Methods

# Findings

# Discussion

- The current measurements are likely influenced by human intervention, i.e. the opening or closing of dams. We could not find a dataset which described these actions so we couldn't account for it. If actions are taken based on the predictions of a model that's trained on this kind of biased data, there is risk of a feedback loop where the model makes predictions as if an intervening action will be made.